Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
# %%bash
# pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf/7B

### Step 1: Load the model

Point model_id to model weight folder

In [2]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="./models_hf/7B"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

# Set the tokenizer's pad token if it's not already set
if tokenizer.pad_token is None:
    if tokenizer.eos_token:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        # If there is no EOS token, add a new pad token (this depends on your tokenizer)
        # This is just an example; your tokenizer might use a different pad token
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})


model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Step 2: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

In [3]:
import json
import pandas as pd
from datasets import Dataset
import pyarrow as pa
import pyarrow.dataset as ds

#path to the json file containing all the input output pairs of all the 5 teams
json_file_path = 'merged_file.json'

# Read and handle UTF-8 BOM if present
with open(json_file_path, 'r', encoding='utf-8-sig') as file:
    json_data = json.load(file)  # Load the file content as JSON
    
# Function to tokenize pairs of questions and answers
def tokenize_pairs(examples):
    # Tokenize inputs and outputs
    tokenized_inputs = tokenizer(examples['input'], padding='max_length', truncation=True, max_length=512)
    tokenized_outputs = tokenizer(examples['output'], padding='max_length', truncation=True, max_length=512)
    # Return a dictionary combining both
    return {**tokenized_inputs, **{'labels': tokenized_outputs['input_ids']}}

# Convert the JSON data into a pandas DataFrame
dataframe = pd.DataFrame(json_data)

# Create a Hugging Face dataset from the pandas DataFrame
dataset = Dataset(pa.Table.from_pandas(dataframe))

tokenized_dataset = dataset.map(tokenize_pairs, batched=True)

Map:   0%|          | 0/436 [00:00<?, ? examples/s]

In [4]:
print(type(tokenized_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [5]:
print("Columns in the dataset:", tokenized_dataset.column_names)
print("Shape of the dataset:", tokenized_dataset.shape)
print("Features of the dataset:", tokenized_dataset.features)


Columns in the dataset: ['input', 'output', 'input_ids', 'attention_mask', 'labels']
Shape of the dataset: (436, 5)
Features of the dataset: {'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


### Step 3: Check base model

Run the base model on an example input:

eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [6]:
from typing import List, Dict

def create_prompts(json_file: str) -> List[Dict]:
    with open(json_file, 'r', encoding='utf-8-sig') as f:
        items = json.load(f)  # Load the entire JSON file as a list of objects
    qa_pairs = [{
        'input': item.get('input'),
        'expected_output': item.get('output')
    } for item in items if item.get('input') is not None and item.get('output') is not None]
    return qa_pairs

In [7]:
import random

# Generate 3 random numbers between 0 and 435
random_input_output_pairs_index = [random.randint(0, 435) for _ in range(3)]
print(random_input_output_pairs_index)

[395, 160, 263, 45, 147, 62, 384, 352, 272, 206]


In [8]:
qa_pairs = create_prompts('merged_file.json')

for index in random_input_output_pairs_index:
    qa_pair = qa_pairs[index]
    eval_prompt = f"""
        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : {qa_pair['input']}
    """

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question :  Which two organizations are collaborating to release guidance on the use of AI in employment decisions to protect against discrimination towards people with disabilities?
    
        User Question :  Which two organizations are collaborating to release guidance on the use of AI in employment decisions to protect against discrimination towards people with disabilities?
        User Question :  Which two organizations are collaborating to release guidance on the use of AI in employment decisions to protect against discrimi

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : How can the AI assurance industry contribute to regulatory compliance and AI system improvement?
    
        User Question : How can the AI assurance industry contribute to regulatory compliance and AI system improvement?
        Answer : The AI assurance industry can contribute to regulatory compliance and AI system improvement by providing independent assessments of AI systems to ensure that they are safe, reliable, and trustworthy.
        AI assurance can help to identify potential risks and issues with AI systems, su

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question :  What specific actions does the EOAI mandate for the Departments of State and Commerce?
    
        User Question :  What specific actions does the EOAI mandate for the Departments of State and Commerce?
        User Question :  What specific actions does the EOAI mandate for the Departments of State and Commerce?
        User Question :  What specific actions does the EOAI mandate for the Departments of State and Commerce?
        User Question :  What specific actions does the EOAI mandate for the Departments of State a

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question :  How does the study suggest students should be involved in AI policy planning and implementation in universities?
    
        User Question :  How does the study suggest students should be involved in AI policy planning and implementation in universities?
        User Question :  How does the study suggest students should be involved in AI policy planning and implementation in universities?
        User Question :  How does the study suggest students should be involved in AI policy planning and implementation in universit

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : What is China’s contribution to the advancement of AI?
    
        User Question : What is the role of AI in the field of healthcare?
    
        User Question : What are the ethical considerations in the use of AI?
    
        User Question : What are the challenges in the use of AI?
    
        User Question : What are the benefits of AI?
    
        User Question : What are the risks of AI?
    
        User Question : What are the regulations and policies in the use of AI?
    
        User Question : What are the

We can see that the base model only repeats the conversation.

### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [11]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"],
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


### Step 5: Define an optional profiler

In [12]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Step 6: Fine tune the model

Here, we fine tune the model for a single epoch which takes a bit more than an hour on a A100.

In [13]:
def formatting_prompts_func(example) -> list:
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [17]:
from transformers import default_data_collator, Trainer, TrainingArguments
from trl import SFTTrainer

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=False,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer = tokenizer,
        #peft_config = lora_config,
        train_dataset=tokenized_dataset,
        #formatting_func=formatting_prompts_func,
        #max_seq_length=512,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:459: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/jet/home/achoudh2/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to fl

Step,Training Loss
10,19.192300
20,2.809000
30,1.728000
40,1.647000


KeyboardInterrupt: 

In [15]:
from torch.utils.data import DataLoader

# Create a DataLoader to check batch shapes
data_loader = DataLoader(tokenized_dataset, batch_size=10, collate_fn=default_data_collator)

# Check the first batch
for batch in data_loader:
    print({k: v.shape for k, v in batch.items()})
    break  # Just look at the first batch


{'input_ids': torch.Size([10, 512]), 'attention_mask': torch.Size([10, 512]), 'labels': torch.Size([10, 512])}


In [31]:
from torch.utils.data import DataLoader

In [32]:
for batch in DataLoader(tokenized_dataset, batch_size=10, collate_fn=default_data_collator):
    print({k: v.shape for k, v in batch.items()})
    break  # Remove or comment out this line to inspect all batches

{'input_ids': torch.Size([10, 512]), 'attention_mask': torch.Size([10, 512]), 'labels': torch.Size([10, 512])}


### Step 7:
Save model checkpoint

In [12]:
model.save_pretrained(output_dir)

/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


### Step 8:
Try the fine tuned model on the same example again to see the learning progress:

In [29]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [28]:
for index in random_input_output_pairs_index:
    qa_pair = qa_pairs[index]
    eval_prompt = f"""
        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : {qa_pair['input']}
    """

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : How can we increase transparency regarding the development and use of AI?
    
        User Question : What are the key principles of AI ethics?
    
        User Question : What are the key principles of AI ethics?
    
        User Question : What are the key principles of AI ethics?
    
        User Question : What are the key principles of AI ethics?
    
        User Question : What are the key principles of AI ethics?
    
        User Question : What are the key principles of AI ethics?
    
        User Question :

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : How many AI use cases did federal agencies report, and what stage are most of these cases in?
    
        User Question : What are the top 5 AI use cases that federal agencies reported?
    
        User Question : What are the top 5 AI use cases that federal agencies reported?
    
        User Question : What are the top 5 AI use cases that federal agencies reported?
    
        User Question : What are the top 5 AI use cases that federal agencies reported?
    
        User Question : What are the top 5 AI use cases t

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : How does the European Union classify AI systems under its AI Act, and what are the implications for "high risk" AI systems?
    
        User Question : What are the key principles of the OECD's AI Principles, and how do they relate to the AI Act?
    
        User Question : What are the key principles of the OECD's AI Principles, and how do they relate to the AI Act?
    
        User Question : What are the key principles of the OECD's AI Principles, and how do they relate to the AI Act?
    
        User Question : Wha

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : Which elected officials have actively supported the Affordable Connectivity Program (ACP)?
    
        User Question : Which elected officials have actively supported the Affordable Connectivity Program (ACP)?
        User Question : Which elected officials have actively supported the Affordable Connectivity Program (ACP)?
        User Question : Which elected officials have actively supported the Affordable Connectivity Program (ACP)?
        User Question : Which elected officials have actively supported the Affordable 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        You are an AI chatbot with an expertise in AI policy.
        Chatbot Name: PolicyAdvisorAI
        Purpose: To provide information and insights about AI regulations, policies, and ethical considerations from around the world, helping users navigate the complex landscape of AI governance.
        Instruction: Your task is to understand the User Question asked by the user carefully and then provide just the answer to the User Question.
        User Question : What opportunity does Vice President Kamala Harris have regarding international AI governance?
    
        User Question : What is the role of the United States in international AI governance?
    
        User Question : What is the role of the United States in international AI governance?
    
        User Question : What is the role of the United States in international AI governance?
    
        User Question : What is the role of the United States in international AI governance?
    
        User Question : What is 